In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pwd

/home/sid/ds_nudge_up/playground


In [3]:
import matplotlib.pyplot as plt

In [4]:
import sys
sys.path += ["../"]

In [5]:
import os
import src.utils as utl
rasterio_config = utl.load_yaml_config("rasterioc_config.yaml")
os.environ.update(rasterio_config)

In [66]:
from src.featurize import *
from src.nn_forward_pass import *

In [78]:
featurization_params = utl.load_yaml_config("featurisation.yaml")
satellite_config = utl.load_yaml_config("satellite_config.yaml")
satellite_config = satellite_config[featurization_params['satellite_image_params']['satellite_name']]
data_sources = utl.load_yaml_config('data_catalog.yaml')

In [72]:
satellite_config

{'resolution': 60,
 'bands': ['B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B11', 'B12'],
 'buffer_distance': 1200,
 'min_image_edge': 90}

In [13]:
points_gdf = utl.load_points_gdf(**data_sources['request_points'])

In [14]:
%%time
points_gdf_with_stac = fetch_image_refs(points_gdf, featurization_params['dask']['n_partitions'], 
                                        featurization_params['satellite_image_params'])

CPU times: user 81.6 ms, sys: 19.3 ms, total: 101 ms
Wall time: 100 ms


In [15]:
points_gdf_with_stac = points_gdf_with_stac.compute()

In [16]:
mini_stac_gdf = points_gdf_with_stac[:10]

In [64]:
data_loader = create_data_loader(mini_stac_gdf, satellite_config, featurization_params['batch_size'])

In [87]:
model = RCF(featurization_params['num_features'], featurization_params['kernel_size'], len(satellite_config['bands']))
device = torch.device('cpu') 

In [79]:
X_features = create_features(data_loader, featurization_params['num_features'], len(mini_stac_gdf), 
                             model, device, satellite_config['min_image_edge'])

  0%|                                                                                             | 0/2 [00:00<?, ?it/s]
0it [00:00, ?it/s]
 50%|██████████████████████████████████████████                                          | 1/2 [04:43<04:43, 283.10s/it]
0it [00:00, ?it/s]
100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [05:35<00:00, 167.69s/it]


In [80]:
X_features

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [83]:
images = next(iter(data_loader))

In [84]:
image = images[0]

In [88]:
mosaiks_features = featurize(image, model, device)